In [24]:
import pandas as pd
import os

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import rmgpy
from rmgpy.molecule import Molecule

In [35]:
working_dir = os.getcwd()
fpath = 'Vuori.csv'

# Read the CSV file
table_df = pd.read_csv(fpath)

# Initialize a list to store ThermoData objects
thermo_library = []
molecule_names = []
adjacency_lists = []
chemical_formulas = []

In [36]:
for index, row in table_df.iterrows():
    # Extract data from each row
    molecule_name = row['Chemical Formula']
    H298 = row['dHf-298K']
    S298 = row['S-298K']
    Cp298 = row['Cp-298K']
    Cp500 = row['Cp-500K']
    Cp1000 = row['Cp-1000K']
    smiles = row['SMILES']

    # Define temperature and Cp data
    Tdata = [298, 500, 1000]
    Cpdata = [Cp298, Cp500, Cp1000]

    # Create RMG Molecule object from SMILES
    m = Molecule().from_smiles(smiles)
    adj_list = m.to_adjacency_list()
    chemical_formula = m.get_formula()

    # Create ThermoData object
    thermo = rmgpy.thermo.ThermoData(
        Tdata=(Tdata, "K"),
        Cpdata=(Cpdata, "J/(mol*K)"),
        H298=(H298, "kJ/mol"),
        S298=(S298, "J/(mol*K)"),
        Cp0=(m.calculate_cp0(), "J/(mol*K)"),  # Use calculated Cp0
        CpInf=(m.calculate_cpinf(), "J/(mol*K)")  # Use calculated CpInf
    )

    # Convert ThermoData to Wilhoit and back to ThermoData
    wilhoit = thermo.to_wilhoit(B=400)
    thermo2 = wilhoit.to_thermo_data()

    # Append to thermo_library
    thermo_library.append(thermo2)
    molecule_names.append(molecule_name)
    adjacency_lists.append(adj_list)
    chemical_formulas.append(chemical_formula)

    # Tmax = 1500

    # T = np.linspace(1, Tmax, 100)
    # Cp = [wilhoit.get_heat_capacity(Ti) for Ti in T]

    # plot Cpdata vs Tdata for thermo and thermo2
    # plt.figure()

    # plt.plot(thermo.Tdata.value_si, thermo.Cpdata.value_si, 'o', label='original')
    # plt.plot(thermo2.Tdata.value_si, thermo2.Cpdata.value_si, 'x', label='converted')
    # plt.plot(T, Cp, label='Wilhoit')
    # plt.hlines(wilhoit.CpInf.value_si, xmin=0, xmax=Tmax, linestyles=':', label='CpInf')
    # plt.plot(0, wilhoit.Cp0.value_si, 'o', label='Cp0')
    # plt.legend()

    # # Adding labels and title
    # plt.xlabel('Temperature (K)')
    # plt.ylabel('Heat Capacity (J/(mol*K))')
    # plt.ylim(bottom = 0)
    # plt.title(f"{chemical_formula}")

    # figure_name = os.path.join(working_dir, "Figures", f"Cp_{chemical_formula}.png")
    # plt.savefig(figure_name)

    # plt.close()



In [50]:
# Write output to Boron_Vuori.py
output_path = os.path.join(working_dir, 'Boron_Vuori.py')

with open(output_path, 'w') as f:
    for i, (molecule_name, adj_list, thermo, chemical_formula) in enumerate(zip(molecule_names, adjacency_lists, thermo_library, chemical_formulas)):
        f.write(f"entry(\n")
        f.write(f"    index = {i},\n")
        f.write(f"    label = '{molecule_name}',\n")
        f.write(f"    molecule = \n")
        f.write(f"\"\"\"\n{adj_list}\"\"\",\n")
        f.write(f"    thermo = ThermoData(\n")
        f.write(f"        Tdata = {thermo.Tdata!r},\n")
        f.write(f"        Cpdata = {thermo.Cpdata!r},\n")
        f.write(f"        H298 = {thermo.H298!r},\n")
        f.write(f"        S298 = {thermo.S298!r},\n")
        f.write(f"        Cp0 = {thermo.Cp0!r},\n")
        f.write(f"        CpInf = {thermo.CpInf!r},\n")
        f.write(f"        E0 = {thermo.E0!r},\n")
        f.write(f"    ),\n")
        f.write(f"    shortDesc = u\"\"\"{chemical_formula}\"\"\",\n")
        f.write(f"    longDesc = u\"\"\"Calculated with W1X-1\"\"\",\n")
        f.write(f")\n\n")